## Direct solve

In [ ]:
using OptimalControl

# Parameters
Cd = 310.
Tmax = 3.5
β = 500.
b = 2.
N = 100
t0 = 0.
r0 = 1.
v0 = 0.
vmax = 0.1
m0 = 1.
mf = 0.6

ocp = Model()

@variable(ocp, tf)
@time(ocp, t ∈ (t0, tf))
@state(ocp, x[3])
@control(ocp, u)

@constraint(ocp, x(0) == [ r0, v0, m0 ], :initial_con)
@constraint(ocp, 0. ≤ u(t) ≤ 1.)
@constraint(ocp, r0 ≤ x[1], :state_con1)
@constraint(ocp, 0 ≤ x[2] ≤ vmax, :state_con2)
@constraint(ocp, m0 ≤ x[3] ≤ mf, :state_con3)

@objective(ocp, r(tf), Max)

function F0(x)
    r, v, m = x
    D = Cd * v^2 * exp(-β*(r-1.))
    F = [ v, -D/m-1.0/r^2, 0. ]
    return F
end

function F1(x)
    r, v, m = x
    F = [ 0., Tmax/m, -b*Tmax ]
    return F
end

function f!(x, u, dx)
    dx[:] = F0(x(t)) + u[1]*F1(x(t))
end

@constraint(ocp, f!(x(t), u(t), ∂x(t)))

sol = solve(ocp)
plot(sol)

## Indirect solve

In [ ]:
# Bang controls
u0(x, p) = 0.
u1(x, p) = 1.

# Computation of singular control of order 1
H0(x, p) = p' * F0(x)
H1(x, p) = p' * F1(x)
H01 = {H0, H1}
H001 = {H0, H01}
H101 = {H1, H01}
us(x, p) = -H001(x, p) / H101(x, p)

# Computation of boundary control
remove_constraint!(ocp, :state_con1)
remove_constraint!(ocp, :state_con3)
@constraint(ocp, x(tf)[3] == mf, :final_con)
g = vmax - constraint(ocp, :state_con2)
ub(x) = -Ad(F0, g)(x) / Ad(F1, g)(x)
μb(x, p) = H01(x, p) / Ad(F1, g)(x)

f0 = flow(ocp, u0)
f1 = flow(ocp, u1)
fs = flow(ocp, us)
fb = flow(ocp, ub, μb)

# Shooting function
function shoot!(x0, p0, t1, t2, t3, tf, s) # B+ S C B0 structure

    x1, p1 = f1(t0, x0, p0, t1)
    x2, p2 = fs(t1, x1, p1, t2)
    x3, p3 = fb(t2, x2, p2, t3)
    xf, pf = f0(t3, x3, p3, tf)
    s[1:3] = x0 - constraint(ocp, :initial_con)
    s[4] = constraint(ocp, :final_con)(t0, x0, tf, xf) - mf
    s[5:6] = pf[1:2] - [ 1.0, 0.0 ]
    s[7] = H1(x1, p1)
    s[8] = H01(x1, p1)
    s[9] = g(x2)
    s[10] = H0(xf, pf) # free tf

end